# CNN

In [1]:
import sys
sys.path.append('/notebooks/MOD2NN')

In [47]:
import os
import json

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from CNN.module.lib.layers import *
tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.8.0
Python: 3.8.11 (default, Aug 16 2021, 12:04:33) 
[Clang 12.0.0 ]


In [37]:
class Real2Complex(tf.keras.layers.Layer):
    def __init__(self, phase=0.):
        super(Real2Complex, self).__init__()
        self.phase = tf.constant(phase, dtype=tf.float32)

    def get_config(self):
        config = super().get_config()
        config.update({
            "phase": self.phase.numpy()
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

    def call(self, inputs):
        return tf.complex(inputs, 0.) * tf.complex(tf.cos(self.phase), tf.sin(self.phase))

In [38]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = np.sqrt(x_train.astype("float32") / 255.0)
x_test = np.sqrt(x_test.astype("float32") / 255.0)

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

In [43]:
def create_model():
    inputs = tf.keras.Input((28,28))
    x = layers.Flatten()(inputs)
    x = Real2Complex()(x)
    x = ComplexDense(10)(x)
    x = MulConjugate()(x)
    x = Real()(x)

    return tf.keras.Model(inputs, x)

In [44]:
model = create_model()
model.get_config()

{'name': 'model_5',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 28, 28),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_12'},
   'name': 'input_12',
   'inbound_nodes': []},
  {'class_name': 'Flatten',
   'config': {'name': 'flatten_11',
    'trainable': True,
    'dtype': 'float32',
    'data_format': 'channels_last'},
   'name': 'flatten_11',
   'inbound_nodes': [[['input_12', 0, 0, {}]]]},
  {'class_name': 'Real2Complex',
   'config': {'name': 'real2_complex_9',
    'trainable': True,
    'dtype': 'float32',
    'phase': 0.0},
   'name': 'real2_complex_9',
   'inbound_nodes': [[['flatten_11', 0, 0, {}]]]},
  {'class_name': 'ComplexDense',
   'config': {'name': 'complex_dense_6',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'use_bias': False},
   'name': 'complex_dense_6',
   'inbound_nodes': [[['real2_complex_9', 0, 0, {}]]]},
  {'class_name': 'MulConjugate',
   'config': {'name': 'mu

In [34]:
model_name = "20221124_1"
path = "/notebooks/"
checkpoint_path = path + "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
path = path + "trained_model/"+ model_name

model = create_model()
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),  # category: sparse_categorical_crossentropy
              metrics=[tf.keras.metrics.BinaryCrossentropy(from_logits=True)]
              )

epochs = 50
early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        min_delta=0.0002,
        patience=2,
    )

def lr_schedule(epoch):
  """
  Returns a custom learning rate that decreases as epochs progress.
  """
  start = 0.2
  end = 0.0001
  n = 50
  a = -1/(n-1) * np.log(end/start)
  x = np.arange(n)
  return np.exp(-a*(epoch-1)) * start
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

# チェックポイントコールバックを作る
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

logdir = os.path.join(path +"logs", model_name)
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

result = model.fit(x_train,
                   x_train,
                   batch_size=64,
                   epochs=epochs,
                   validation_data=(x_val, y_val),
                   callbacks=[cp_callback, tensorboard_callback]
                   )

model.save(path)

df = pd.DataFrame(result.history)
df.to_csv(path + "/history.csv")



with open(path + "/config.json", 'w') as f:
    json.dump(model.get_config(), f, indent=4)